# Basic *pyodbc* driver

Using only *pyodbc* driver. Can list all tables.

In [ ]:
import pyodbc
server = 'hraap-euw-prod-asq01.database.windows.net'
database = 'hraap-euw-db01'
username ='brian.linkletter@nokia.com'
Authentication='ActiveDirectoryInteractive'
driver= '{ODBC Driver 18 for SQL Server}'
conn = pyodbc.connect('DRIVER='+driver+
                      ';SERVER='+server+
                      ';PORT=1433;DATABASE='+database+
                      ';UID='+username+
                      ';AUTHENTICATION='+Authentication
                      )

print(conn)

In [ ]:
import pprint
cursor = conn.cursor()
print(dir(cursor.tables()))

# for row in cursor.tables():
#     print(row.table_name)

statement = """
SELECT TABLE_CATALOG, TABLE_SCHEMA, TABLE_NAME, COLUMN_NAME, COLUMN_DEFAULT
FROM hraap-euw-db01.INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = N'Snapshot P24 Last Close';
"""

print(statement)

#cursor.execute(statement)

# cursor.execute("SELECT * in table [Snapshot P24 Last Close];") 
# row = cursor.fetchone()
# x = 0
# while x < 5: 
#     print(row[0])
#     row = cursor.fetchone()
#     x += 1

In [ ]:
query = cursor.execute('Select TOP 5 * FROM [employment_details] table [Snapshot Non P24 Last Close];')

In [ ]:
cursor.execute("SELECT @@version;") 
row = cursor.fetchone() 
print(row)
# while row: 
#     print(row[0])
#     row = cursor.fetchone()

# SQLAlchemy

Can list schemas but cannot list tables

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import URL
from azure.identity import InteractiveBrowserCredential
import pyodbc

server_name = 'hraap-euw-prod-asq01.database.windows.net'
database_name = 'hraap-euw-db01'
user_name = 'brian.linkletter@nokia.com'
connection_string = (
    f"Driver={{ODBC Driver 18 for SQL Server}};"
    f"Server={server_name};"
    f"Database={database_name};"
    f"Authentication=ActiveDirectoryInteractive;"
    f"UID={user_name};PWD=''"
)

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={connection_string}")


In [2]:
from sqlalchemy import MetaData
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

# Base = automap_base()
# Base.prepare(autoload_with=engine)

metadata = MetaData(schema="employment_details")
metadata.reflect(engine)

# tables = ['Snapshot P24 Last Close']
# metadata.reflect(engine, only=tables)

Base = automap_base(metadata=metadata)
Base.prepare()

In [3]:
metadata.tables.keys()

dict_keys([])

In [ ]:
print(metadata.reflect(engine, schema="employment_details"))
#metadata.tables["employment_details.Snapshot P24 Last Close"]

In [ ]:
print(Base.metadata.tables)

# SQLAlchemy inspect

Inspect() function lists schema (run after the login above)

In [ ]:
print(engine)
from sqlalchemy import inspect
inspector = inspect(engine)

In [ ]:
#dir(inspector)
db_list = inspector.get_schema_names()
print(db_list)

In [ ]:
inspector.get_table_names(schema='employment_details')

In [ ]:
statement = "SELECT TOP 10 * FROM [employment_details.Snapshot P24 Last Close]"
P24close = pd.read_sql_query(statement, engine)

In [ ]:
P24data = pd.read_sql_table(table_name='Snapshot P24 Last Close', con=engine)